## Dangers of Application User Input:

### User Input Potential Dangers 
- #### User input can contain unexpected characters and symbols
- #### Users can provide input shorter or longer than the length you expected
- #### Users can deduce semantics or URL parameters POST requests and replay or manipulate all of them
- #### Users can and will intentionally attack applications

### In safe implementations: 
- #### **User Input ALWAYS** has to be constrained in length, unless specified otherwise 
- #### **User Input ALWAYS** has to be sanitized with RegEx or code logic
- #### Any sanitation faults must be logged 
- #### Any file reading must be wrapped in exception handling blocks at all tiers of the application depth from the Front-End to the Back-End. 
    
### On the front end, RegEx and Input Limiting/Masking are the most common sanitation methods of user input. Use session management and URL parameter protection methods to prevent web-requests replay and URL parameters manipulation. On the back-end, the use of Parametrized and Type-Locked Database queries and stored procedures help realize the defense-in-depth concept.  

In [1]:
import re   # re is part of 'Lib/re' in standard distribution that is installed with Python. 
            # There is also 'regex' library that has very similar API and can be installed separately
import time

email_or_user = '^(?:\w+|\w+([+\.-_]?\w+)*@\w+([\.-]?\w+)*(\.[a-zA-z]{2,4})+)$'

IS_CORRECT_USER_NAME = re.compile(email_or_user)


while True:
    # !!! NEVER Remove Sleep - it breaks Input  - the famous quirk of Python Server in a Python Notebook
    time.sleep(0.5) # In executing the snippet from command-line the sleep is not needed    
    maybe_user = input('Try To Enter User Name : ')
    if re.fullmatch(IS_CORRECT_USER_NAME, maybe_user):
        print(f'\n\t+++ User Name "{maybe_user}" is compliant +++')
    else:
        print(f'\n\t!!! User Name "{maybe_user}" is NOT ALLOWED !!!')
    if maybe_user.lower() in ['exit', 'quit', 'break', 'done', 'kill', 'x']:
        break



	+++ User Name "1234567890" is compliant +++

	!!! User Name "Owl-Grey_like Tea" is NOT ALLOWED !!!

	+++ User Name "x" is compliant +++


## Does it Mean That File Input Is Safe?

### File Input Potential Dangers
- #### Files can be made malformed or become corrupted
- #### File size, line length, encoding, decoding, and/or any other properties assumed or expected by the code may be wrong
- #### File inputs can be intentionally changed to attack applications and point to broken or intentionally modified files
- #### The data read from files can be modified to present danger at the place of use in the code

### In safe implementations, **file input ALWAYS** has to be verified to comply with the assumptions made in the code and with the developer's expectations.

### Example of a SIMPLE INTERACTIVE Python implementation that reads a YAML file into a dict below:

In [25]:
import yaml
def read_settings(settings_file:str)->dict: 
    # Here we initialize the dict with the 
    # DEFAULT VALUES in case the YAML file could not be read!
    settings_dict={'database': './db/BACKUP_DB.db', 'attempts': 10} 
    stream = None
    try:
        with open(settings_file, "r") as stream:
            settings_dict = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(f'yaml.YAMLError:\n{exc}\n')
    except Exception as general_ex:
        print(f'General Exception:\n{general_ex}\n')
    finally:
        if stream:
            stream.close()
    return settings_dict


info_1 = read_settings("../Tests/si/set/settings.yaml") # Read a good file
info_2 = read_settings("../Tests/si/set/settings.yamlX") # read non-existing file
info_3 = read_settings("../Tests/si/set/settings-broken.yaml") # read structurally incorrect file

print(f'\nInfo 1 = {info_1}')
print(f'\nInfo 2 = {info_2}')
print(f'\nInfo 3 = {info_3}')


General Exception:
[Errno 2] No such file or directory: '../Tests/si/set/settings.yamlX'

yaml.YAMLError:
while scanning a simple key
  in "../Tests/si/set/settings-broken.yaml", line 3, column 1
could not find expected ':'
  in "../Tests/si/set/settings-broken.yaml", line 4, column 2


Info 1 = {'DB_NAME': '~/SI_DB.db', 'TABLE_NAME': 'USERS_TOPIC_1', 'MAX_FAILED': 5}

Info 2 = {'database': './db/BACKUP_DB.db', 'attempts': 10}

Info 3 = {'database': './db/BACKUP_DB.db', 'attempts': 10}


#### The application should produce error messages like the following:

```
General Exception:
[Errno 2] No such file or directory: '../Tests/si/set/settings.yamlX'
```

```
yaml.YAMLError:
while scanning a simple key
  in "../Tests/si/set/settings-broken.yaml", line 3, column 1
could not find expected ':'
  in "../Tests/si/set/settings-broken.yaml", line 4, column 2
```

### Though you would want to prevent phone and web applications from spilling information about your back-end or middle-tier file system, database type, database structure, server configurations, and/or other "helpful information" that feels useful to a developer, but surely will expose knowledge about your system's internal structure.  While that information is not useful for the average user, it exposes necessary information to a potential attacker.

### This information, including where the failure occurred, would be better preserved in a local log file. The user should be given a generic message stating that the operation could not be accomplished. Below is an implementation of the previous code, now with logging functionality.

In [1]:
import yaml
import logging
import io
from datetime import datetime

def read_settings(settings_file:str)->dict: 
    # Here we initialize the dict with the 
    # DEFAULT VALUES in case the YAML file could not be read!
    settings_dict={'database': './db/BACKUP_DB.db', 'attempts': 10} 
    stream = None
    status = -100
    try:
        with open(settings_file, "r") as stream:
            settings_dict = yaml.safe_load(stream)
            status = 1
    except yaml.YAMLError as exc:
        log_message = (f'yaml.YAMLError:\n{exc}\n')
        app_logger.error(log_message, exc_info=True)
        status = -1
    except Exception as general_ex:
        log_message = (f'General Exception:\n{general_ex}\n')
        app_logger.exception(log_message, exc_info=True)
        status = -10
    finally:
        if stream:
            stream.close()
        if status<0:
            app_logger.error(f'Failed attempt to read YAML file [{settings_file}] with status:{status}')
        else:
            app_logger.info(f'Successfully read YAML file [{settings_file}] with status:{status}')
    return settings_dict

def configure_logger():
    LOG_FORMAT = (  f'\n\n!!!{"="*80}\n!!!=> %(asctime)s \n'
                    f'\tLogged by: %(name)s\t@Level: %(levelname)s:\n'
                    f'\tIn function: [%(funcName)s]\t@Line No: %(lineno)d\n'
                    f'\t%(message)s\n!!!{"="*80}')
    #
    # Lambda solution with microseconds if needed from stack overflow https://stackoverflow.com/questions/50873446/python-logger-output-dates-in-is8601-format
    # logging.Formatter.formatTime = (lambda self, record, datefmt=None: datetime.datetime.fromtimestamp(record.created, datetime.timezone.utc).astimezone().isoformat(sep="T",timespec="milliseconds"))
    #
    datetime_format_ISO = '%Y-%m-%dT%H:%M:%S%z' # # %f does not work, despite the Python specs - hence dropped microseconds
    menacing_format =  '%Y-%m-%dT%H:%M:%S.%f %A the %'

    logging.basicConfig(
        filename = './status.log',
        level=logging.DEBUG, 
        filemode='a',
        format = LOG_FORMAT,
        datefmt=datetime_format_ISO
        ) # Handlers array can contain multiple streams
    return logging.getLogger(__name__)

app_logger = configure_logger()

info_1 = read_settings("../Tests/si/set/settings.yaml") # Read a good file
info_2 = read_settings("../Tests/si/set/settings.yamlX") # read non-existing file
info_3 = read_settings("../Tests/si/set/settings-broken.yaml") # read structurally incorrect file

print(f'\nInfo 1 = {info_1}')
print(f'\nInfo 2 = {info_2}')
print(f'\nInfo 3 = {info_3}')


Info 1 = {'DB_NAME': '~/SI_DB.db', 'TABLE_NAME': 'USERS_TOPIC_1', 'MAX_FAILED': 5}

Info 2 = {'database': './db/BACKUP_DB.db', 'attempts': 10}

Info 3 = {'database': './db/BACKUP_DB.db', 'attempts': 10}
